In [ ]:
import torch
from torchvision import transforms
from torch.utils.data import Dataset,DataLoader
import os
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.model_selection import KFold
import sys
sys.path.append('../input/autoaugment')
from autoaugment import ImageNetPolicy
import torch.optim as optim

In [ ]:
#resnet34
import torch.nn as nn
import torch
 
#18和34层残差结构（具备实线残差结构功能和虚线残差结构功能）
class BasicBlock(nn.Module):
    expansion = 1 #对应残差结构中卷积核个数有没有发生变化。1是1倍意思，即都一样。
 
    # downsample对应虚线的残差结构
    def __init__(self, in_channel, out_channel, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channel, out_channels=out_channel,
                               kernel_size=3, stride=stride, padding=1, bias=False)
        #注意之前的卷积层不要bias
        self.bn1 = nn.BatchNorm2d(out_channel)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels=out_channel, out_channels=out_channel,
                               kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channel)
        self.downsample = downsample
 
    def forward(self, x):
        identity = x
        if self.downsample is not None:
            identity = self.downsample(x)
 
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
 
        out = self.conv2(out)
        out = self.bn2(out)
 
        out += identity
        out = self.relu(out)
 
        return out
 
 
#50层及以上的残差结构（具备实线残差结构功能和虚线残差结构功能）
class Bottleneck(nn.Module):
    expansion = 4
 
    def __init__(self, in_channel, out_channel, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channel, out_channels=out_channel,
                               kernel_size=1, stride=1, bias=False)  # squeeze channels
        self.bn1 = nn.BatchNorm2d(out_channel)
        # -----------------------------------------
        self.conv2 = nn.Conv2d(in_channels=out_channel, out_channels=out_channel,
                               kernel_size=3, stride=stride, bias=False, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channel)
        # -----------------------------------------
        self.conv3 = nn.Conv2d(in_channels=out_channel, out_channels=out_channel*self.expansion,
                               kernel_size=1, stride=1, bias=False)  # unsqueeze channels
        self.bn3 = nn.BatchNorm2d(out_channel*self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
 
    def forward(self, x):
        identity = x
        if self.downsample is not None:
            identity = self.downsample(x)
 
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
 
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
 
        out = self.conv3(out)
        out = self.bn3(out)
 
        out += identity
        out = self.relu(out)
 
        return out
 
 
# block为 BasicBlock(nn.Module)或Bottleneck(nn.Module)
# blocks_num：列表参数,代表残差结构的个数，如[3,4,6,3]、[2,2,2,2]
# include_top=True方便在resnet上搭建更复杂的结构。默认就是True
class ResNet(nn.Module):
 
    def __init__(self, block, blocks_num, num_classes=1000, include_top=True):
        super(ResNet, self).__init__()
        self.include_top = include_top
        self.in_channel = 64
 
        self.conv1 = nn.Conv2d(3, self.in_channel, kernel_size=7, stride=2,
                               padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(self.in_channel)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, blocks_num[0])
        self.layer2 = self._make_layer(block, 128, blocks_num[1], stride=2)
        self.layer3 = self._make_layer(block, 256, blocks_num[2], stride=2)
        self.layer4 = self._make_layer(block, 512, blocks_num[3], stride=2)
        if self.include_top:
            #采用自适应平均池化，不管输入是什么维度，输出的HW都将是1*1
            self.avgpool = nn.AdaptiveAvgPool2d((1, 1))  # output size = (1, 1)
            self.fc = nn.Linear(512 * block.expansion, num_classes)
 
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
 
    # 搭建残差结构的函数
    def _make_layer(self, block, channel, block_num, stride=1):
        downsample = None
        if stride != 1 or self.in_channel != channel * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channel, channel * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(channel * block.expansion))
 
        layers = []
        layers.append(block(self.in_channel, channel, downsample=downsample, stride=stride))
        self.in_channel = channel * block.expansion
 
        for _ in range(1, block_num):
            layers.append(block(self.in_channel, channel))
 
        return nn.Sequential(*layers) # 将list转为非关键字参数传入
 
 
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
 
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
 
        if self.include_top:
            x = self.avgpool(x)
            x = torch.flatten(x, 1)
            x = self.fc(x)
 
        return x
 
 
def resnet34(num_classes=1000, include_top=True):
    return ResNet(BasicBlock, [3, 4, 6, 3], num_classes=num_classes, include_top=include_top)
 
 
def resnet101(num_classes=1000, include_top=True):
    return ResNet(Bottleneck, [3, 4, 23, 3], num_classes=num_classes, include_top=include_top)

def resnet152(num_classes=1000, include_top=True):
    return ResNet(Bottleneck, [3, 8, 36, 3], num_classes=num_classes, include_top=include_top)

In [ ]:
#data preprocessing
data = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')
data = data.drop(index=(data.loc[(data['labels']=='healthy')].index)) #不使用healthy数据
all_img_names:list = data["image"].values.tolist() #冒号后起注释作用

label_num2str = {0: 'powdery_mildew',
                 1: 'scab',
                 2: 'complex',
                 3: 'frog_eye_leaf_spot',
                 4: 'rust'}

label_str2num = {'powdery_mildew': 0,
                 'scab': 1,
                 'complex': 2,
                 'frog_eye_leaf_spot': 3,
                 'rust': 4}  
     
all_numeric_labels = []     
for row_idx,row in data.iterrows():    
    labels_list = row['labels'].split(" ")
    numeric_label_list = [label_str2num[each] for each in labels_list if each != 'healthy']
    numeric_label_numpy = np.array(numeric_label_list)*0.9+0.05 #label_smoothing
    numeric_label_list = numeric_label_numpy.tolist()
    all_numeric_labels.append(numeric_label_list)  
data.insert(2,'numeric_labels',all_numeric_labels)

all_img_labels_ts = []

for tmp_lb in all_numeric_labels:
    tmp_label = torch.zeros([5],dtype=torch.float)
    for idx in tmp_lb:
        tmp_label[int(idx)] = 1.0 
    all_img_labels_ts.append(tmp_label)
k_fold = KFold(n_splits = 6,shuffle = True,random_state = 77) #六折交叉验证
  


In [ ]:
#Focal_loss
class FocalLoss(nn.Module):
    """
    The focal loss for fighting against class-imbalance
    """
    def __init__(self, alpha=1, gamma=2):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = 1e-12  # prevent training from Nan-loss error
        self.cls_weights = torch.tensor([[0.3648, 0.0813, 0.2184, 0.1066, 0.2290]],dtype=torch.float, requires_grad=False, device=torch.device("cuda:0"))
        self.lb_smooth = 0.1

    def forward(self, logits, target):
        """
        logits & target should be tensors with shape [batch_size, num_classes]
        """
        probs = torch.sigmoid(logits)
        one_subtract_probs = 1.0 - probs
        # add epsilon
        probs_new = probs + self.epsilon
        one_subtract_probs_new = one_subtract_probs + self.epsilon
        # calculate focal loss
        target = torch.abs(target - self.lb_smooth)
        log_pt = target * torch.log(probs_new) + (1.0 - target) * torch.log(one_subtract_probs_new)
        pt = torch.exp(log_pt)
        focal_loss = -1.0 * (self.alpha * (1 - pt) ** self.gamma) * log_pt
        focal_loss = focal_loss * self.cls_weights
        return torch.mean(focal_loss)

In [ ]:
#F1 score 

class MyF1Score(Metric):
    def __init__(self, threshold: float = 0.5, dist_sync_on_step=False):
        super().__init__(dist_sync_on_step=dist_sync_on_step)
        self.threshold = threshold
        self.add_state("tp", default=torch.tensor(0), dist_reduce_fx="sum")
        self.add_state("fp", default=torch.tensor(0), dist_reduce_fx="sum")
        self.add_state("fn", default=torch.tensor(0), dist_reduce_fx="sum")

    def update(self, preds: torch.Tensor, target: torch.Tensor):
        assert preds.shape == target.shape
        preds_str_batch = self.num_to_str(torch.sigmoid(preds))
        target_str_batch = self.num_to_str(target)
        tp, fp, fn = 0, 0, 0
        for pred_str_list, target_str_list in zip(preds_str_batch, target_str_batch):
            for pred_str in pred_str_list:
                if pred_str in target_str_list:
                    tp += 1
                if pred_str not in target_str_list:
                    fp += 1

            for target_str in target_str_list:
                if target_str not in pred_str_list:
                    fn += 1
        self.tp += tp
        self.fp += fp
        self.fn += fn

    def compute(self):
        f1 = 2.0 * self.tp / (2.0 * self.tp + self.fn + self.fp)
        return f1
    
    def num_to_str(self, ts: torch.Tensor) -> list:
        batch_bool_list = (ts > self.threshold).detach().cpu().numpy().tolist()
        batch_str_list = []
        for one_sample_bool in batch_bool_list:
            lb_str_list = [self.label_num2str[lb_idx] for lb_idx, bool_val in enumerate(one_sample_bool) if bool_val]
            if len(lb_str_list) == 0:
                lb_str_list = ['healthy']
            batch_str_list.append(lb_str_list)
        return batch_str_list

In [ ]:
#dataset
class PlantDataset(Dataset):
    def __init__(self,img_path,img_name:list,img_label:list,transform = None):
        self.img_path = img_path
        self.img_label = img_label
        self.img_name = img_name
        self.transform = transform
        
    def __len__(self):
        return len(self.img_name)
    
    def __getitem__(self,index):
        path = os.path.join(self.img_path,self.img_name[index])
        img = Image.open(path).convert('RGB')
        img = self.transform(img)
        img_label = self.img_label[index]
        return img,img_label

In [ ]:
#data_transform
data_transform = {
        "train": transforms.Compose([transforms.RandomResizedCrop(380),
                                     transforms.RandomHorizontalFlip(),
                                     ImageNetPolicy(),
                                     transforms.ToTensor(),
                                     transforms.Normalize([0.46141568, 0.6165156, 0.3905568], [0.15031955, 0.1329983, 0.172767])]),
        "val": transforms.Compose([transforms.Resize(412), #将图像最小的边缩放到256.
                                   transforms.CenterCrop(380),
                                   transforms.ToTensor(),
                                   transforms.Normalize([0.46141568, 0.6165156, 0.3905568], [0.15031955, 0.1329983, 0.172767])])}

In [ ]:
#train
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = resnet34() #实例化网络，注意：此时没有传入参数，默认是1000分类
    
# load pretrain weights
# download url: https://download.pytorch.org/models/resnet34-333f7ec4.pth
model_weight_path = "../input/resnet34-pre/resnet34-pre.pth"
assert os.path.exists(model_weight_path), "file {} does not exist.".format(model_weight_path)
missing_keys, unexpected_keys = net.load_state_dict(torch.load(model_weight_path), strict=False)
# for param in net.parameters():
#     param.requires_grad = False
# change fc layer structure
in_channel = net.fc.in_features
net.fc = nn.Linear(in_channel, 5) #由于花分类是5类，所以重新赋值（默认1000分类）

net.to(device)

#loss_function = FocalLoss(class_num=12,alpha=(0.983,0.792,0.990,0.729,0.933,0.995,0.891,0.994,0.993,0.718,0.960,0.988),use_alpha=(True))
#loss_function = nn.CrossEntropyLoss()
loss_function = FocalLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0003)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=2,
             verbose=False, threshold=1e-4, threshold_mode='rel',
             cooldown=0, min_lr=0, eps=1e-8)   

best_acc = 0.0
save_path = './resNet34_multi.pth'

for epoch in range(3):
  # get image names and labels   
    for fold_idx, (train_indices, valid_indices) in enumerate(k_fold.split(all_img_names)):
        fold_train_img_names = [all_img_names[idx] for idx in train_indices]
        fold_valid_img_names = [all_img_names[idx] for idx in valid_indices]
        fold_train_img_labels = [all_img_labels_ts[idx] for idx in train_indices]
        fold_valid_img_labels = [all_img_labels_ts[idx] for idx in valid_indices]
        # dataset
        train_dataset = PlantDataset('../input/plant-pathology-2021-fgvc8/train_images', fold_train_img_names, fold_train_img_labels, data_transform['train'])
        valid_dataset = PlantDataset('../input/plant-pathology-2021-fgvc8/train_images', fold_valid_img_names, fold_valid_img_labels, data_transform['val'])
        # dataloader
        train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=8, drop_last=True)
        valid_loader = DataLoader(valid_dataset, batch_size=8, shuffle=False, num_workers=8)
        
        val_num = len(valid_dataset)

        # train
        net.train()
        running_loss = 0.0
        for step, data in enumerate(train_loader, start=0):
            images, labels = data
            optimizer.zero_grad()
            logits = net(images.to(device))
            loss = loss_function(logits, labels.to(device))
            loss.backward()
            optimizer.step()
            # print statistics
            running_loss += loss.item()
            # print train process
            rate = (step+1)/len(train_loader)
            a = "*" * int(rate * 50)
            b = "." * int((1 - rate) * 50)
            print("\rtrain loss: {:^3.0f}%[{}->{}]{:.4f}".format(int(rate*100), a, b, loss), end="")
        print()

        # validate
        net.eval()
        val_num = len(valid_dataset)
        acc = 0.0  # accumulate accurate number / epoch
        with torch.no_grad():
            for val_data in valid_loader:
                val_images, val_labels = val_data
                outputs = net(val_images.to(device))  # eval model only have last output layer
                
                '''
                outputs = torch.sigmoid(outputs).cuda()
                # loss = loss_function(outputs, test_labels)
                for i in range(outputs.shape[0]):
                    for j in range(outputs.shape[1]):
                        if outputs[i][j] > 0.68997:
                            outputs[i][j] = 1
                        else:outputs[i][j] = 0  
                '''
                #predict_y = outputs
                #acc += (predict_y == val_labels.to(device)).sum().item()
            #val_accurate = acc / val_num
            val_accurate = MyF1Score(preds = outputs, target = val_labels).cuda()
            if val_accurate > best_acc:
                best_acc = val_accurate
                torch.save(net.state_dict(), save_path)
            print('[epoch %d] train_loss: %.3f  test_accuracy: %.3f' %
                  (epoch + 1, running_loss / step, val_accurate))
        scheduler.step(val_accurate)
    print("\r第%d个epoch的学习率：%f" % (epoch, optimizer.param_groups[0]['lr']))
print('Finished Training')

In [ ]:
import torch
from model import resnet34
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt
import json
 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
 
data_transform = transforms.Compose(
    [transforms.Resize(412),
     transforms.CenterCrop(380),
     transforms.ToTensor(),
     transforms.Normalize([0.46141568, 0.6165156, 0.3905568], [0.15031955, 0.1329983, 0.172767])])
 
# create model
model = resnet34(num_classes=5)
# load model weights
model_weight_path = "./resNet34.pth"
model.load_state_dict(torch.load(model_weight_path, map_location=device))
model.eval()
with torch.no_grad():
    # predict class
    output = torch.squeeze(model(img))
    predict = torch.sigmoid(output)
    predict_cla = torch.argmax(predict).numpy()
print(class_indict[str(predict_cla)], predict[predict_cla].numpy())
plt.show()


In [ ]:
from sklearn.model_selection import KFold
a = [1,2,3,4,5]
k_fold = KFold(n_splits = 5,shuffle = True,random_state = 77) #六折交叉验证
for fold_idx, (train_indices, valid_indices) in enumerate(k_fold.split(a)):
    print(fold_idx,'train:',train_indices,'valid:',valid_indices)

In [ ]:
import torch
a = torch.tensor([0.022,0.002,0.001,0.86,0.98])
b = torch.sigmoid(a)
print(b)
